## Contador de repetição utilizando MediaPipe e OpenCV
Este projeto demonstra como criar um contador de repetições para exercícios usando as bibliotecas MediaPipe e OpenCV. Com isso, o objetivo é rastrear pontos-chave do corpo humano (como ombros, cotovelos e pulsos) para calcular ângulos de articulação e, assim, determinar o início e o fim de um movimento, como uma flexão de braço (bíceps curl).

## 1. Instalação e Importação de Bibliotecas
Primeiro, será importado as bibliotecas necessárias para o projeto, como o MediaPipe para detecção de pose, OpenCV para processar o vídeo da webcam e o NumPy será usado para cálculos matemáticos.

In [5]:
#Importação das bibliotecas e key dependencies
import cv2
import mediapipe as mp
import numpy as np
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose #Pose estimation models

## 2. Função para Cálculo de Ângulo
Nesta seção, será definido a funçõe que calcularão o ângulo entre as articulações.

In [6]:
#Função para calcular qualquer angulação entre 3 articulações do corpo
def calculate_angle (a, b, c):
    a = np.array(a)
    b = np.array(b)
    c = np.array(c)

    radians = np.arctan2(c[1] - b[1], c[0] - b[0]) - np.arctan2(a[1] - b[1], a[0] - c[0])
    angle = np.abs(radians * 180. / np.pi)

    if angle > 180:
        angle = 360 - angle
    return angle


## 3. Detecção de Pose em Tempo Real
Configuração do fluxo principal do projeto, que captura vídeo da webcam e usa o MediaPipe para detectar os pontos-chave do corpo em tempo real. Além disso, os landmarks de pose são extraídos e desenhados sobre o vídeo para visualização.

In [7]:
#Acionando a webcam para servir como feed para o MediaPipe
cap = cv2.VideoCapture(0)

#Criando as variáveis do contador de repetições
counter = 0
stage = None

#Aplicando o setup do MediaPipe
with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
    while cap.isOpened():
        ret, frame = cap.read()

        #Mudando a coloração da imagem de BGR para RGB
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False

        #Faz a detecção das poses em formato de "SolutionOutputs"
        results = pose.process(image)

        #Mudando a coloração da imagem de RGB para BGR, já que o OpenCV só aceita imagem em BGR
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

        #Extraindo os pontos de referências
        try:
            landmarks = results.pose_landmarks.landmark #variável landmark será em formato de dicionário

            #Pegando as coordenadas das principais articulações
            shoulder = [landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x, landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y]
            elbow = [landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].x, landmarks[mp_pose.PoseLandmark.LEFT_ELBOW.value].y]
            wrist = [landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].x, landmarks[mp_pose.PoseLandmark.LEFT_WRIST.value].y]
            left_hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x, landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]

            #Calculando o angulo
            angle = calculate_angle(shoulder, elbow, wrist)
            angle_hip2elbow = calculate_angle(left_hip, shoulder, elbow)

            #Realizando a contagem das repetições
            if angle > 160:
                stage = "baixo"
            if angle < 30 and stage == "baixo":
                stage = "cima"
                counter += 1
                print(counter)

            #Visualizando o ângulo no frame
            cv2.putText(image, str(angle), tuple(np.multiply(elbow, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)
            cv2.putText(image, str(angle_hip2elbow), tuple(np.multiply(shoulder, [640, 480]).astype(int)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA)

        except:
            pass

        #Colocando o contador de repetições na tela
        #Setup do retângulo
        cv2.rectangle(image, (0, 0), (250, 73), (245, 117, 16), -1)

        #Texto que vai indicar o número de repetições
        cv2.putText(image, "REPS: ", (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, str(counter), (10,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        #Texto que vai indicar o estado do braço
        cv2.putText(image, "ESTADO: ", (90,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
        cv2.putText(image, stage, (90,60), cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)

        '''Renderizando a detecção para sair do formato "SolutionOutputs" e ir para um dicionário de coordenadas (x,y,z,visibilidade)
        com o results.pose_landmarks das articulações capturadas, já o mp_pose.POSE_CONNECTIONS seria a conexão entre
        as articulações'''
        mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                  mp_drawing.DrawingSpec(color=(245,117,66),thickness=2,circle_radius=2),
                                  mp_drawing.DrawingSpec(color=(245, 66, 230), thickness=2, circle_radius=2)
                                  )

        #Mostrando as coordenadas de cada articulação e as ligaões entre as articulações
        #print(results.pose_landmarks)
        #print(mp_pose.POSE_CONNECTIONS)

        cv2.imshow('MediaPipe Feed', image)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

1
2
